In [1]:
import os
import re
import sys
import time
import tweepy
import json
import datetime 
import collections
import numpy as np
import pandas as pd
import dateutil.parser
from collections import Counter
from tweepy import OAuthHandler
from apscheduler.schedulers.blocking import BlockingScheduler

In [ ]:
def poster():

    log = pd.read_csv('Login.csv')
    consumerKey = log['key'][0]
    consumerSecret = log['key'][1]
    accesToken = log['key'][2]
    accesTokenSecret = log['key'][3]
    bearer_token = log['key'][4]
    authenticate = tweepy.OAuthHandler(consumerKey, consumerSecret)
    authenticate.set_access_token(accesToken, accesTokenSecret)
    api = tweepy.API(authenticate, wait_on_rate_limit = True)
    client = tweepy.Client(consumer_key = consumerKey, consumer_secret = consumerSecret, access_token = accesToken, access_token_secret = accesTokenSecret)

    #Mexico woeid = 23424900
    #CDMX woeid = 116545
    #US woeid = 23424977
    #NewYork woeid = 2459115
    #London woeid = 44418
    #World woeid = 1

    woeid = 23424977
    trend_name_lst = []
    trend_count_lst = []
    trends = api.get_place_trends(id = woeid)

    for value in trends:
        for trend in value['trends']:
            trend_name_lst.append(trend['name'])
            trend_count_lst.append(trend['tweet_volume'])

    df_trend = pd.DataFrame(trend_name_lst, columns=['trend'])
    df_trend['count'] = trend_count_lst
    df_trend['date'] = datetime.date.today()
    df_trend.to_csv('trend.csv')
    df_trend_asc = df_trend.sort_values("count", ascending=False)
    hashtag_lst = []

    for trend in df_trend_asc['trend']:

        hashtag = re.findall(r'#\w+', trend)
        hashtag_lst.append(hashtag)

    df_hashtags = pd.DataFrame(hashtag_lst, columns=['hashtags'])
    df_hashtags_clean = df_hashtags.dropna()
    df_hashtags_clean

    monkey_tweet_text = 'Want the best nudes with no BS? Follow the Monkey: https://t.me/Th3StashNetwork'
    space = ' '
    monkey_hashtags_list = []

    for element in df_hashtags_clean['hashtags']:
        monkey_hashtags_list.append(element)

    monkey_hashtags = ' '.join([str(item) for item in monkey_hashtags_list ])
    monkey_tweet = monkey_tweet_text + space + '\n' + monkey_hashtags

    response = client.create_tweet(text = monkey_tweet)
    print(response)


scheduler = BlockingScheduler()
scheduler.add_job(poster, 'interval', seconds=10)
scheduler.start()